In [1]:
import functools as ft
from datetime import datetime
from itertools import combinations

import numpy as np
import pandas as pd

from src.prisma import prisma

In [2]:
async def startup():
    await prisma.connect()

In [3]:
async def shutdown():
    await prisma.disconnect()

In [4]:
await startup()

# Information Indices

In [5]:
def convert_str_timestamp_to_timestamp(timestamp_str: str, timestamp_format: str = "%Y-%m-%dT%H:%M:%S.%fZ") -> int:
    timestamp = datetime.timestamp(datetime.strptime(timestamp_str, timestamp_format))

    return timestamp

In [6]:
async def get_player_damaged_player_event(event_id: str) -> dict:
    player_damage_player_raw_data = await prisma.playerdamagedplayerevent.find_first(
        where={
            'eventId': event_id,
        }
    )

    player_damage_player_data = player_damage_player_raw_data.dict()

    return player_damage_player_data

In [7]:
async def get_player_killed_player_event(event_id: str) -> dict:
    player_killed_player_raw_data = await prisma.playerkilledplayerevent.find_first(
        where={
            'eventId': event_id,
        }
    )

    player_killed_player_data = player_killed_player_raw_data.dict()

    return player_killed_player_data

In [8]:
async def get_rounds_by_match_id(match_id: str) -> list:
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'events': True
        },
    )

    match_rounds_data = []
    for match_round_raw_data in match_rounds_raw_data:
        match_rounds_data.append(match_round_raw_data.dict())

    return match_rounds_data

In [9]:
async def get_rounds_pdp_pkp_data_by_match_id(match_id: str) -> dict:
    rounds_by_match_id = await get_rounds_by_match_id(match_id)

    # rounds_data = {
    #     round_id: {
    #         pdp: {
    #             actor: {
    #                 target: [(d_d, t_d),...]
    #             }
    #         }
    #         pkp: {
    #             actor: {
    #                 target: (true, t_k)
    #             }
    #         }
    #     }
    # }
    rounds_data = {}
    for round in rounds_by_match_id:
        round_id = round["roundId"]
    
        events = {
            "pdp": {},
            "pkp": {},
        }
        for event in round["events"]:
            event_id = event["eventId"]
            timestamp = event["timestamp"]
            
            if event["eventType"] == "PLAYER_DAMAGED_PLAYER":
                player_damaged_player_event = await get_player_damaged_player_event(event_id)
                
                actor = player_damaged_player_event["actorPlayerId"]
                target = player_damaged_player_event["targetPlayerId"]
                damage = player_damaged_player_event["damageDealt"]
    
                if actor in events["pdp"]:
                    if target in events["pdp"][actor]:
                        events["pdp"][actor][target] += [(damage, timestamp)]
                    else:
                        events["pdp"][actor] = {
                            target: [(damage, timestamp)]
                        }
                else:
                    events["pdp"][actor] = {
                        target: [(damage, timestamp)]
                    }
            elif event["eventType"] == "PLAYER_KILLED_PLAYER":
                player_killed_player_event = await get_player_killed_player_event(event_id)
                
                actor = player_killed_player_event["actorPlayerId"]
                target = player_killed_player_event["targetPlayerId"]
    
                if actor in events["pkp"]:
                    if target in events["pkp"][actor]:
                        continue
                    else:
                        events["pkp"][actor][target] = (True, timestamp)
                else:
                    events["pkp"][actor] = {
                        target: (True, timestamp)
                    }
                
        rounds_data[round_id] = events

    return rounds_data

In [10]:
async def get_rounds_pdp_pkp_data_by_match_ids(match_ids: list[str]) -> dict:
    rounds_pdp_pkp_data_by_match_ids = {}
    for match_id in match_ids:
        rounds_pdp_pkp_data_by_match_id = await get_rounds_pdp_pkp_data_by_match_id(match_id)

        rounds_pdp_pkp_data_by_match_ids.update(rounds_pdp_pkp_data_by_match_id)

    return rounds_pdp_pkp_data_by_match_ids

In [11]:
async def get_player_damage_kill_information_index_by_match_ids(match_ids: list[str]) -> pd.DataFrame:
    rounds_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)
    
    player_damage_kill_information_index_raw_data = {}
    for round_id, round_data in rounds_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                kill_damage_delta_t = []
                if actor_id in round_data["pdp"]:
                    if target_id in round_data["pdp"][actor_id]:
                        for damage_dealt, damage_timestamp in round_data["pdp"][actor_id][target_id]:
                            kill_damage_delta_t.append(
                                convert_str_timestamp_to_timestamp(kill_timestamp)-convert_str_timestamp_to_timestamp(damage_timestamp)
                            )
                
                if actor_id in player_damage_kill_information_index_raw_data:
                    player_damage_kill_information_index_raw_data[actor_id] += kill_damage_delta_t
                else:
                    player_damage_kill_information_index_raw_data[actor_id] = kill_damage_delta_t
    
    players_damage_kill_information_index = []
    for player, data in player_damage_kill_information_index_raw_data.items():
        mean_t_k_diff_t_d = np.mean(data)
        std_t_k_diff_t_d = np.std(data)
        player_damage_kill_information_index = std_t_k_diff_t_d / mean_t_k_diff_t_d
        
        players_damage_kill_information_index += [
            {
                "playerId": player,
                "player_damage_kill_information_index": player_damage_kill_information_index,
            }
        ]
    
    players_damage_kill_information_index = pd.DataFrame(players_damage_kill_information_index)
    
    return players_damage_kill_information_index

In [12]:
async def get_player_kill_kill_information_index_by_match_ids(match_ids: list[str]) -> pd.DataFrame:
    rounds_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)

    player_kill_kill_information_index_raw_data = {}
    for round_id, round_data in rounds_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            kill_timestamps = []
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                kill_timestamps.append(kill_timestamp)
    
            if len(kill_timestamps) > 1:
                idxs = list(combinations(range(len(kill_timestamps)), 2))
    
                kill_timestamps_delta_t = []
                for (i, j) in idxs:
                    kill_timestamps_delta_t.append(
                        abs(convert_str_timestamp_to_timestamp(kill_timestamps[i])-convert_str_timestamp_to_timestamp(kill_timestamps[j]))
                    )
    
                if actor_id in player_kill_kill_information_index_raw_data:
                    player_kill_kill_information_index_raw_data[actor_id] += kill_timestamps_delta_t
                else:
                    player_kill_kill_information_index_raw_data[actor_id] = kill_timestamps_delta_t
            else:
                continue

    players_kill_kill_information_index = []
    for player, data in player_kill_kill_information_index_raw_data.items():
        mean_t_k_diff_t_k = np.mean(data)
        std_t_k_diff_t_k = np.std(data)
        player_kill_kill_information_index = std_t_k_diff_t_k / mean_t_k_diff_t_k
        
        players_kill_kill_information_index += [
            {
                "playerId": player,
                "player_kill_kill_information_index": player_kill_kill_information_index,
            }
        ]
    
    players_kill_kill_information_index = pd.DataFrame(players_kill_kill_information_index)
    
    return players_kill_kill_information_index

# General Metrics

In [13]:
# Match
## Round
### Player
#### Kills - Average
#### Damage Dealt - Average
#### Headshots - Average
#### % Opponent Team Killed - Average
#### Ace - Count
#### Clutch - Count
#### Last Man Standing Time - Average
#### Estimated Player View Time - Average

In [14]:
async def get_players_rounds_stats_by_match_id(match_id: str) -> list:
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'playerRoundStats': True
        },
    )

    match_rounds_players_stats = []
    for match_round_raw_data in match_rounds_raw_data:
        match_round_raw_data = match_round_raw_data.dict()

        match_round_players_stats = []
        for match_round_player_stats in match_round_raw_data["playerRoundStats"]:
            match_round_player_stats["matchId"] = match_round_raw_data["matchId"]
    
            match_round_players_stats.append(match_round_player_stats)

        match_rounds_players_stats += match_round_players_stats

    return match_rounds_players_stats

In [15]:
async def get_players_rounds_stats_by_match_ids(matmatch_ids: list[str]) -> list:
    matchs_rounds_players_stats = []
    for match_id in match_ids:
        match_rounds_players_stats = await get_players_rounds_stats_by_match_id(match_id)

        matchs_rounds_players_stats += match_rounds_players_stats

    return matchs_rounds_players_stats

In [16]:
async def get_players_rounds_stats_by_match_ids_df(matmatch_ids: list[str]) -> pd.DataFrame:
    rounds_stats_by_match_ids = await get_players_rounds_stats_by_match_ids(match_ids)

    df_rounds_stats_by_match_ids = pd.DataFrame(rounds_stats_by_match_ids)

    return df_rounds_stats_by_match_ids

# Player Stats

In [17]:
async def get_players_stats_df(match_ids: list[str]) -> pd.DataFrame:
    # General metrics.
    df_rounds_stats_by_match_ids = await get_players_rounds_stats_by_match_ids_df(match_ids)

    columns_average = [
        "estimatedViewTime", "LastPersonStandingTime", "damageDealt", "percentageOppTeamkilled"
    ]
    columns_count = [
        "Ace", "Clutch", "kills", "headshots"
    ]

    df_players_stats_averages = df_rounds_stats_by_match_ids[["playerId"] + columns_average].groupby(["playerId"]).mean().reset_index()
    df_players_stats_counts = df_rounds_stats_by_match_ids[["playerId"] + columns_count].groupby(["playerId"]).sum().reset_index()
    df_players_rounds_count = df_rounds_stats_by_match_ids[["playerId", "roundId"]].groupby(["playerId"]).nunique().reset_index()
    df_players_matches_count = df_rounds_stats_by_match_ids[["playerId", "matchId"]].groupby(["playerId"]).nunique().reset_index()

    # Information indices.
    players_damage_kill_information_index = await get_player_damage_kill_information_index_by_match_ids(match_ids)
    players_kill_kill_information_index = await get_player_kill_kill_information_index_by_match_ids(match_ids)

    # Player stats.
    dfs_players_stats = [
        df_players_stats_averages, df_players_stats_counts, df_players_rounds_count, 
        df_players_matches_count, players_damage_kill_information_index, players_kill_kill_information_index
    ]

    df_players_stats = ft.reduce(lambda left, right: pd.merge(left, right, on="playerId", how="outer"), dfs_players_stats)

    df_players_stats.rename(
        columns={
            "estimatedViewTime": "averageEstimatedPlayerViewTime",
            "LastPersonStandingTime": "averageLastManStandingTime",
            "damageDealt": "averageDamageDealt", 
            "percentageOppTeamkilled": "averagePercentOpponentTeamKilled",
            "Ace": "totalNumberOfAces", 
            "Clutch": "totalNumberOfClutches", 
            "kills": "totalNumberOfKills", 
            "headshots": "totalNumberOfHeadshots",
            "roundId": "numberOfRoundsPlayed",
            "matchId": "numberOfMatchesPlayed",
            "player_damage_kill_information_index": "playerDamageKillInformationIndex",
            "player_kill_kill_information_index":  "playerKillKillInformationIndex",
        }, 
        inplace=True
    )

    df_players_stats["averageKillsPerRound"] = df_players_stats["totalNumberOfKills"] / df_players_stats["numberOfRoundsPlayed"]
    df_players_stats["averageHeadshotsPerRound"] = df_players_stats["totalNumberOfHeadshots"] / df_players_stats["numberOfRoundsPlayed"]
    
    return df_players_stats

In [18]:
match_ids = ["c5c7500c-6db8-45c8-b6ab-2be01b401ef7", "c7c65bec-8c54-4dea-afcd-c057320e1f8f"]

In [19]:
players_stats_df = await get_players_stats_df(match_ids)

In [20]:
players_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   playerId                          10 non-null     object 
 1   averageEstimatedPlayerViewTime    10 non-null     float64
 2   averageLastManStandingTime        10 non-null     float64
 3   averageDamageDealt                10 non-null     float64
 4   averagePercentOpponentTeamKilled  10 non-null     float64
 5   totalNumberOfAces                 10 non-null     int64  
 6   totalNumberOfClutches             10 non-null     int64  
 7   totalNumberOfKills                10 non-null     int64  
 8   totalNumberOfHeadshots            10 non-null     int64  
 9   numberOfRoundsPlayed              10 non-null     int64  
 10  numberOfMatchesPlayed             10 non-null     int64  
 11  playerDamageKillInformationIndex  10 non-null     float64
 12  playerKillK

In [21]:
players_stats_df

,playerId,averageEstimatedPlayerViewTime,averageLastManStandingTime,averageDamageDealt,averagePercentOpponentTeamKilled,totalNumberOfAces,totalNumberOfClutches,totalNumberOfKills,totalNumberOfHeadshots,numberOfRoundsPlayed,numberOfMatchesPlayed,playerDamageKillInformationIndex,playerKillKillInformationIndex,averageKillsPerRound,averageHeadshotsPerRound
0,76561198026088156,12.257368,2.273170,92.622642,0.181132,0,0,48,23,53,2,1.769723,0.881074,0.905660,0.433962
1,76561198030545997,11.299407,2.400698,85.150943,0.135849,0,0,36,28,53,2,5.081932,1.423909,0.679245,0.528302
2,76561198031908632,10.659005,1.152321,71.132075,0.116981,0,0,31,17,53,2,5.501956,1.341342,0.584906,0.320755
3,76561198042070812,13.177189,5.024717,66.716981,0.124528,0,1,33,19,53,2,2.986396,0.916287,0.622642,0.358491
4,76561198047198945,12.321158,2.736396,80.094340,0.135849,0,1,36,21,53,2,2.409053,1.021063,0.679245,0.396226
5,76561198059614402,11.698010,1.633491,83.094340,0.173585,0,0,46,22,53,2,1.593481,1.129134,0.867925,0.415094
6,76561198070876270,10.822352,1.441075,65.037736,0.105660,0,0,28,19,53,2,1.506099,0.868748,0.528302,0.358491
7,76561198102263776,12.206192,2.960472,61.547170,0.128302,0,1,34,8,53,2,2.881179,1.061578,0.641509,0.150943
8,76561198350362302,10.136335,1.412887,81.132075,0.147170,0,1,39,26,53,2,1.737125,1.180512,0.735849,0.490566
9,76561198365697700,11.311223,1.235453,95.962264,0.184906,0,0,49,22,53,2,2.951527,0.766109,0.924528,0.415094


# Push Player Stats to DB

In [22]:
async def post_player_stats_to_db(players_stats_df: pd.DataFrame):
    async with prisma.batch_() as batcher:
        players_stats = players_stats_df.to_dict("records")

        for player_stat in players_stats:
            batcher.playercard.create(player_stat)

        await batcher.commit()

In [23]:
await post_player_stats_to_db(players_stats_df)

# Key Moments Identifier

In [24]:
async def get_players_damage_kill_moments_df(match_ids: list[str], damage_kill_delta_t_threshold: float = 0.5)  -> pd.DataFrame:
    rounds_pdp_pkp_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)
    
    players_damage_kill_moments_t = {}
    for round_id, round_data in rounds_pdp_pkp_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                if actor_id in round_data["pdp"]:
                    if target_id in round_data["pdp"][actor_id]:
                        n = len(round_data["pdp"][actor_id][target_id])
    
                        if n <= 1:
                            continue
                        
                        delta_ts = []
                        for i in reversed(range(1, n)):
                            _, damage_timestamp_i = round_data["pdp"][actor_id][target_id][i]
                            _, damage_timestamp_i_min_1 = round_data["pdp"][actor_id][target_id][i-1]
    
                            delta_t = convert_str_timestamp_to_timestamp(damage_timestamp_i) - convert_str_timestamp_to_timestamp(damage_timestamp_i_min_1)
    
                            if delta_t > damage_kill_delta_t_threshold:
                                break
    
                            delta_ts.append(delta_t)
    
                        damage_kill_moment_t = sum(delta_ts)
    
                        if damage_kill_moment_t <= 0.0:
                            continue
    
                        if actor_id in players_damage_kill_moments_t:
                            players_damage_kill_moments_t[actor_id] += [damage_kill_moment_t]
                        else:
                            players_damage_kill_moments_t[actor_id] = [damage_kill_moment_t]

    players_damage_kill_moments = []
    for player_id, damage_kill_moments_t in players_damage_kill_moments_t.items():
        players_damage_kill_moments += [
            {
                "playerId": player_id,
                "damageKillKeyMomentCount": len(damage_kill_moments_t),
                "damageKillKeyMomentTotalTime": np.sum(damage_kill_moments_t),
            }
        ]
    
    players_damage_kill_moments = pd.DataFrame(players_damage_kill_moments)

    return players_damage_kill_moments

In [25]:
await get_players_damage_kill_moments_df(match_ids)

,playerId,damageKillKeyMomentCount,damageKillKeyMomentTotalTime
0,76561198365697700,18,4.637
1,76561198047198945,9,3.642
2,76561198350362302,13,4.275
3,76561198026088156,16,4.639
4,76561198031908632,10,3.005
5,76561198059614402,11,3.788
6,76561198070876270,8,1.878
7,76561198030545997,9,2.836
8,76561198102263776,6,1.740
9,76561198042070812,10,4.205


In [26]:
async def get_players_kill_kill_moments_df(match_ids: list[str], kill_kill_delta_t_threshold: float = 2.0)  -> pd.DataFrame:
    rounds_pdp_pkp_data = await get_rounds_pdp_pkp_data_by_match_ids(match_ids)
    
    players_kill_kill_moments_t = {}
    for round_id, round_data in rounds_pdp_pkp_data.items():
        # Round
        for actor_id, target_data in round_data["pkp"].items():
            # Actor
            kill_timestamps = []
            for target_id, (_, kill_timestamp) in target_data.items():
                # Target
                kill_timestamps.append(kill_timestamp)
    
            n = len(kill_timestamps)
    
            if n <= 1:
                continue
            
            delta_ts = []
            for i in reversed(range(1, n)):
                kill_timestamp_i = kill_timestamps[i]
                kill_timestamp_i_min_1 = kill_timestamps[i-1]
    
                delta_t = convert_str_timestamp_to_timestamp(kill_timestamp_i) - convert_str_timestamp_to_timestamp(kill_timestamp_i_min_1)
    
                if delta_t > kill_kill_delta_t_threshold:
                    break
    
                delta_ts.append(delta_t)
    
            kill_kill_moment_t = sum(delta_ts)
    
            if kill_kill_moment_t <= 0.0:
                continue
    
            if actor_id in players_kill_kill_moments_t:
                players_kill_kill_moments_t[actor_id] += [kill_kill_moment_t]
            else:
                players_kill_kill_moments_t[actor_id] = [kill_kill_moment_t]

    players_kill_kill_moments = []
    for player_id, kill_kill_moments_t in players_kill_kill_moments_t.items():
        players_kill_kill_moments += [
            {
                "playerId": player_id,
                "killKillKeyMomentCount": len(kill_kill_moments_t),
                "killKillKeyMomentTotalTime": np.sum(kill_kill_moments_t),
            }
        ]
    
    players_kill_kill_moments = pd.DataFrame(players_kill_kill_moments)

    return players_kill_kill_moments

In [27]:
await get_players_kill_kill_moments_df(match_ids)

,playerId,killKillKeyMomentCount,killKillKeyMomentTotalTime
0,76561198070876270,1,1.531
1,76561198365697700,5,6.841
2,76561198059614402,4,7.101
3,76561198026088156,1,1.989
4,76561198350362302,3,3.547
5,76561198031908632,2,2.344
6,76561198030545997,1,1.295
7,76561198042070812,2,2.656
8,76561198047198945,2,3.896


In [ ]:
player_id = "76561198026088156"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

    if player_id in rounds_pdp_pkp_data[round_id]["pdp"]:
        print("="*10, "Damages", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pdp"][player_id])

In [ ]:
player_id = "76561198102263776"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

    if player_id in rounds_pdp_pkp_data[round_id]["pdp"]:
        print("="*10, "Damages", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pdp"][player_id])

In [ ]:
player_id = "76561198059614402"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])

In [ ]:
player_id = "76561198102263776"
for round_id, rounds_data in rounds_pdp_pkp_data.items():
    print("="*10, round_id, "="*10)
    if player_id in rounds_pdp_pkp_data[round_id]["pkp"]:
        print("="*10, "Kills", "="*10)
        print(rounds_pdp_pkp_data[round_id]["pkp"][player_id])